# **Urban Simulation Exercise 2**

### **Author:** Noam Gal

In [3]:
# import relevant packages and functions
from random import choice, randint, shuffle, choices
import random
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import linregress

# **Part A**

In [5]:
class Building:    
    # initialize building
    def __init__(self, geometry):
        '''input: geometry (Shapley geometry object)'''
        self.geometry = geometry # store Shapely geometry object as attribute
        
        self.agents = [] # stores agents located within the cell
        self.crimes = 0 # tracks the number of crimes committed in the cell
    
    def distance(self, b):
        '''input: b (Building object)
            returns the Euclidean distance between the object and b'''
        return self.geometry.distance(b.geometry)


class Agent:
    def __init__(self, a_type, building, budget):
        '''inputs:
            a_type (str): type of agent - resident/criminal,
            building (Building object): building in which the agent starts the simulation
            budget (int): resources at the disposal of the agent
            '''
        self.a_type = a_type
        self.building = building
        self.budget = budget
        
        building.agents.append(self) # locate agent inside the cell
    
    def resident_step(self):
        '''step method for resident agents - moves to a random new building'''
        self.budget += 1 # resources increase with every step
        new_building = self.building # set current building as new building
        while new_building == self.building: # loop to make sure a new building is selected
            new_building = choice(model.buildings) # randomly choose a building
        return new_building
        
    def criminal_utility(self, building, max_agents):
        '''Computes the utility the agent derives from the input building, 
        based on distance and number of agents in the building.
        Inputs: building (Building object) - the building for which to compute utility, 
        max_agents (int) - the maximal number of agents present in one building (required for normalization)'''
        d = self.building.distance(building) # compute distance of current position from distance
        if d > self.budget: # if distance if greater than budget - utility is zero
            return 0
        
        d_utility = 1 - d / self.budget # normalize distance by budget and compute the inverse
        c_utility = len(building.agents) / max_agents # normalize number of agents by maximal number of agents
        # compute utility as a linear combination of distance utility and crime utility, using weight
        utility = model.weight * c_utility + (1 - model.weight) * d_utility
        return utility
    
    def criminal_step(self):
        '''step method for criminal agents - 
        move to a random building based on probabilities, randomly select an agent and steal 1 resource point'''
        buildings = [b for b in model.buildings if b != self.building] # get all buildings but current
        max_agents = max([len(b.agents) for b in buildings]) # find the maximal number of agents per building
        utilities = [self.criminal_utility(b, max_agents) for b in buildings] # compute utility value per building
        if sum(utilities) > 0: # choices require that the sum of weights would be greater than 0
            new_building = choices(buildings, utilities)[0] # randomly (with weights based on probabilities) choose one building
        else: # if utilities sum to zero
            new_building = choice(buildings) # choose a random building
            
        if new_building.agents: # if there are any agents in the building (empty list==False)
            agent = choice(new_building.agents) # randomly choose an agent present in the building
            if agent.budget > 0: # if the agent has any resources - steal one unit
                agent.budget -= 1 # remove one unit of resources from selected agent
                self.budget += 1 # add one unit to current agent
                new_building.crimes += 1 # increase the new building's crime counter
    
        return new_building
    
    def step(self):
        '''General step method for both types of agents:
            locates new building and makes the required changes'''
        if self.a_type == 'resident': # check agent type and activate step method
            new_building = self.resident_step()
        else:
            new_building = self.criminal_step()
        
        self.building.agents.remove(self) # remove current agent from current building
        new_building.agents.append(self) # add current agent to new building
        self.building = new_building # move current agent to new building
        

class Model:
    def __init__(self, residents_num, criminal_num, weight, bldgs):
        '''inputs: residents_num (int) - number of resident agents,
        criminal_num (int) - number of criminal agents,
        weight (float) - weight parameter of crime factor in utility function
        bldgs (GeoDataFrame object) - geodata on buildings'''
        
        self.weight = weight
        
        # create a list of building objects
        self.buildings = [Building(row.geometry) for idx, row in bldgs.iterrows()]
        # create a 1d Array of Agent objects  (length==resident_num) located in a random building and with a random budget
        self.residents = [Agent('resident', choice(self.buildings), randint(50, 800)) for i in range(residents_num)]
        # create a 1d Array of Agent objects  (length==criminal_num) located in a random building and with budget==50
        self.criminals = [Agent('criminal', choice(self.buildings), 50) for i in range(criminal_num)]
    
    def simulate(self):
        '''Run 30 iterations and return the total number of crimes, maximal number of crimes per building,
        and percent of buildings where no crimes were registered througout the simulation'''
        for i in range(30):
            agents = self.residents + self.criminals # merge the lists of agents into one
            shuffle(agents) # randomly reorder the merged list
            for agent in agents: # execute step function per agent
                agent.step()
        
        # generate outputs - get the count of crimes per building
        crimes = [b.crimes for b in self.buildings]
        total_crimes = sum(crimes) # total number of crimes committed throughout the simulation
        max_crimes = max(crimes) # get the counter value for the building with the most crimes committed
        # compute the percent of buildings where no crime was committed
        no_crimes = 100 * len([c for c in crimes if c==0]) / len(self.buildings)
        
        return [total_crimes, max_crimes, no_crimes]
        

criminal_num = 40
weight = 0.5
buildings = gpd.read_file('/Users/noamgal/Downloads/bldgs_points(1)/bldgs_points.shp')
model = Model(200-criminal_num, criminal_num, weight, buildings)
model.simulate()

[443, 8, 70.64439140811456]

In [6]:
# parameters part A
residents_num = 160
criminal_num = 40
weights = [0.25, 0.5, 0.75]
# create dataframe for Part A results
partA_df = pd.DataFrame(columns=['residents_num', 'criminal_num', 'weight', 'result_total', 'result_max', 'result_percent'])

# Iterates through each of the three sets of parameters requested
for w in weights:
    result_total = []
    result_max = []
    result_percent = []
    # runs the model 5 times
    for simulation in range(5): 
        model = Model(residents_num, criminal_num, w, buildings)
        results = model.simulate()
        #print(results)
        result_total.append(results[0])
        result_max.append(results[1])
        result_percent.append(results[2])
    #print(f'The average total for weight {w} is:',np.mean(result_total))
    #print(f'The average max for weight {w} is:',np.mean(result_max))
    #print(f'The average percent for weight {w} is:',np.mean(result_percent))
    
    # Averages the results and adds output to the dataframe
    new_row = {
        'residents_num': residents_num,
        'criminal_num': criminal_num,
        'weight': w,
        'result_total': np.mean(result_total),
        'result_max': np.mean(result_max),
        'result_percent': np.mean(result_percent)}
    partA_df.loc[len(partA_df)] = new_row
# Run line below to export results
#partA_df.to_csv('SimEx2-partA.csv')

# **Part B**

In [8]:
class Building:    
    # initialize building
    def __init__(self, geometry):
        '''input: geometry (Shapley geometry object)'''
        self.geometry = geometry # store Shapely geometry object as attribute
        
        self.agents = [] # stores agents located within the cell
        self.crimes = 0 # tracks the number of crimes committed in the cell
    
    def distance(self, b):
        '''input: b (Building object)
            returns the Euclidean distance between the object and b'''
        return self.geometry.distance(b.geometry)


class Agent:
    def __init__(self, a_type, building, budget):
        '''inputs:
            a_type (str): type of agent - resident/criminal,
            building (Building object): building in which the agent starts the simulation
            budget (int): resources at the disposal of the agent
            '''
        self.a_type = a_type
        self.building = building
        self.budget = budget
        
        building.agents.append(self) # locate agent inside the cell
    
    def resident_step(self):
        '''step method for resident agents - moves to a random new building in which friend agents are located'''
        self.budget += 1 # resources increase with every step
        # find all buildings in which connected agents are present
        candidates = [a.building for a in nx.generators.ego_graph(model.network, self, radius=3) 
                      if a.building != self.building]
        if len(candidates) == 0: # if there are none, find all buildings except current
            candidates = [b for b in model.buildings if b != self.building] 
        new_building = choice(candidates) # choose new building
        return new_building
        
    def criminal_utility(self, building, max_agents):
        '''Computes the utility the agent derives from the input building, 
        based on distance and number of agents in the building.
        Inputs: building (Building object) - the building for which to compute utility, 
        max_agents (int) - the maximal number of agents present in one building (required for normalization)'''
        d = self.building.distance(building) # compute distance of current position from distance
        if d > self.budget: # if distance if greater than budget - utility is zero
            return 0
        
        d_utility = 1 - d / self.budget # normalize distance by budget and compute the inverse
        c_utility = len(building.agents) / max_agents # normalize number of agents by maximal number of agents
        # compute utility as a linear combination of distance utility and crime utility, using weight
        utility = model.weight * c_utility + (1 - model.weight) * d_utility
        return utility
    
    def criminal_step(self):
        '''step method for criminal agents - 
        move to a random building based on probabilities, randomly select an agent and steal 1 resource point'''
        buildings = [b for b in model.buildings if b != self.building] # get all buildings but current
        max_agents = max([len(b.agents) for b in buildings]) # find the maximal number of agents per building
        utilities = [self.criminal_utility(b, max_agents) for b in buildings] # compute utility value per building
        if sum(utilities) > 0: # choices require that the sum of weights would be greater than 0
            new_building = choices(buildings, utilities)[0] # randomly (with weights based on probabilities) choose one building
        else: # if utilities sum to zero
            new_building = choice(buildings) # choose a random building
            
        if new_building.agents: # if there are any agents in the building (empty list==False)
            agent = choice(new_building.agents) # randomly choose an agent present in the building
            if agent.budget > 0: # if the agent has any resources - steal one unit
                agent.budget -= 1 # remove one unit of resources from selected agent
                self.budget += 1 # add one unit to current agent
                new_building.crimes += 1 # increase the new building's crime counter
    
        return new_building
    
    def step(self):
        '''General step method for both types of agents:
            locates new building and makes the required changes'''
        if self.a_type == 'resident': # check agent type and activate step method
            new_building = self.resident_step()
        else:
            new_building = self.criminal_step()
        
        self.building.agents.remove(self) # remove current agent from current building
        new_building.agents.append(self) # add current agent to new building
        self.building = new_building # move current agent to new building
        

class Model:
    def __init__(self, residents_num, criminal_num, weight, bldgs, connections):
        '''inputs: residents_num (int) - number of resident agents,
        criminal_num (int) - number of criminal agents,
        weight (float) - weight parameter of crime factor in utility function
        bldgs (GeoDataFrame object) - geodata on buildings
        connections (int) - number of other agents an agent knows'''
        
        self.weight = weight
        
        # create a list of building objects
        self.buildings = [Building(row.geometry) for idx, row in bldgs.iterrows()]
        # create a 1d Array of Agent objects  (length==resident_num) located in a random building and with a random budget
        self.residents = [Agent('resident', choice(self.buildings), randint(50, 800)) for i in range(residents_num)]
        # create a 1d Array of Agent objects  (length==criminal_num) located in a random building and with budget==50
        self.criminals = [Agent('criminal', choice(self.buildings), 50) for i in range(criminal_num)]
        
        # inititate social network
        self.network = nx.Graph()
        
        self.network.add_nodes_from(self.residents) # add resident agents to network
        
        for a in self.network.nodes: # for every resident agent = for every node
            candidates = [f for f in self.network.nodes if f != a] # find all agents that are not a
            distances = [a.building.distance(f.building) for f in candidates] # compute distance from agents
            sorted_distances = sorted(distances) # sort distances from lowest to highest
            ranks = [sorted_distances.index(d) for d in distances] # get a rank score for each distance
            # choose only agents whose rank is lower than connections
            friends = [candidates[i] for i in range(len(candidates)) if ranks[i] < connections]
            for f in friends:
                self.network.add_edge(a, f) # add an edge to the network between agent and friend
    
    def simulate(self):
        '''Run 30 iterations and return the total number of crimes, maximal number of crimes per building,
        and percent of buildings where no crimes were registered througout the simulation'''
        for i in range(30):
            agents = self.residents + self.criminals # merge the lists of agents into one
            shuffle(agents) # randomly reorder the merged list
            for agent in agents: # execute step function per agent
                agent.step()
        
        # generate outputs - get the count of crimes per building
        crimes = [b.crimes for b in self.buildings]
        total_crimes = sum(crimes) # total number of crimes committed throughout the simulation
        max_crimes = max(crimes) # get the counter value for the building with the most crimes committed
        # compute the percent of buildings where no crime was committed
        no_crimes = 100 * len([c for c in crimes if c==0]) / len(self.buildings)
        return [total_crimes, max_crimes, no_crimes]
        

criminal_num = 40
weight = 0.5
buildings = gpd.read_file('/Users/noamgal/Downloads/bldgs_points(1)/bldgs_points.shp')
connections = 3
model = Model(200-criminal_num, criminal_num, weight, buildings, connections)
model.simulate()

[121, 17, 93.07875894988067]

In [9]:

# Parameters part B
residents_num = 160
criminal_num = 40
weights = [0.25, 0.5, 0.75]
connections = [1,2,3]

# Create dataframe for Part B results
partB_df = pd.DataFrame(columns=['residents_num', 'criminal_num', 'weight', 'connections', 'result_total', 'result_max', 'result_percent'])

# Run part B
for c in connections:
    # For each set of assignment parameters
    for w in weights:
        result_total = []
        result_max = []
        result_percent = []
        # runs five times for each parameter set and adds results to a list
        for run in range(5):
            model = Model(residents_num, criminal_num, w, buildings, c)
            results = model.simulate()
            #print(results)
            result_total.append(results[0])
            result_max.append(results[1])
            result_percent.append(results[2])
            
        #print(f'The average total for weight {w} and connection {c} is:',np.mean(result_total))
        #print(f'The average max for weight {w} and connection {c} is:',np.mean(result_max))
        #print(f'The average percent for weight {w} and connection {c} is:',np.mean(result_percent))
        
        # Averages the results for each set of parameters and adds to a dataframe
        new_row = {
            'residents_num': residents_num,
            'criminal_num': criminal_num,
            'weight': w,
            'connections': c,
            'result_total': np.mean(result_total),
            'result_max': np.mean(result_max),
            'result_percent': np.mean(result_percent)}
        partB_df.loc[len(partB_df)] = new_row
        
# Export dataframe of results
#partB_df.to_csv('SimEx2-partB.csv')

## Statistical Analysis of Results for Part B

In [11]:
# Define a function to perform regression and print comparisons
def perform_regression(df, x_col, y_col):
    X = df[x_col].values
    y = df[y_col].values
    slope, intercept, r_value, p_value, std_err = linregress(X, y)
    # Print regression results
    print(f"{x_col} vs {y_col}: R-squared = {r_value**2:.2f}, p-value = {p_value:.4f}")

In [12]:
# List of predictor columns
predictors = ['weight', 'connections']

# List of result columns
results = ['result_total', 'result_max', 'result_percent']

# Perform regression between each predictor and each result column
for predictor in predictors:
    for result in results:
        perform_regression(partB_df, predictor, result)

weight vs result_total: R-squared = 0.21, p-value = 0.2126
weight vs result_max: R-squared = 0.58, p-value = 0.0168
weight vs result_percent: R-squared = 0.09, p-value = 0.4358
connections vs result_total: R-squared = 0.64, p-value = 0.0092
connections vs result_max: R-squared = 0.21, p-value = 0.2153
connections vs result_percent: R-squared = 0.75, p-value = 0.0025


In [34]:
partB_df.corr()

,residents_num,criminal_num,weight,connections,result_total,result_max,result_percent
residents_num,NaN,NaN,NaN,NaN,NaN,NaN,NaN
criminal_num,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weight,NaN,NaN,1.000000e+00,-3.700743e-17,0.460167,0.763004,-0.298186
connections,NaN,NaN,-3.700743e-17,1.000000e+00,-0.802845,0.457803,0.866960
result_total,NaN,NaN,4.601673e-01,-8.028450e-01,1.000000,-0.081250,-0.980853
result_max,NaN,NaN,7.630042e-01,4.578025e-01,-0.081250,1.000000,0.216968
result_percent,NaN,NaN,-2.981861e-01,8.669597e-01,-0.980853,0.216968,1.000000


# **Part C**

In [15]:
class Building:    
    # initialize building
    def __init__(self, geometry):
        '''input: geometry (Shapley geometry object)'''
        self.geometry = geometry # store Shapely geometry object as attribute
        
        self.agents = [] # stores agents located within the cell
        self.crimes = 0 # tracks the number of crimes committed in the cell
    
    def distance(self, b):
        '''input: b (Building object)
            returns the Euclidean distance between the object and b'''
        return self.geometry.distance(b.geometry)


class Agent:
    def __init__(self, a_type, building, budget):
        '''inputs:
            a_type (str): type of agent - resident/criminal,
            building (Building object): building in which the agent starts the simulation
            budget (int): resources at the disposal of the agent
            '''
        self.a_type = a_type
        self.building = building
        self.budget = budget
        
        building.agents.append(self) # locate agent inside the cell
        
    def resident_step(self):
        '''step method for resident agents - moves to a random new building in which friend agents are located'''
        self.budget += 1 # resources increase with every step
        # find all buildings in which connected agents are present
        candidates = [a.building for a in nx.generators.ego_graph(model.network, self, radius=3) 
                      if a.building != self.building]
        if len(candidates) == 0: # if there are none, find all buildings except current
            candidates = [b for b in model.buildings if b != self.building] 
        new_building = choice(candidates) # choose new building
        return new_building
        
    def criminal_utility(self, building, max_agents):
        '''Computes the utility the agent derives from the input building, 
        based on distance and number of agents in the building.
        Inputs: building (Building object) - the building for which to compute utility, 
        max_agents (int) - the maximal number of agents present in one building (required for normalization)'''
        d = self.building.distance(building) # compute distance of current position from distance
        if d > self.budget: # if distance if greater than budget - utility is zero
            return 0
        
        d_utility = 1 - d / self.budget # normalize distance by budget and compute the inverse
        c_utility = len(building.agents) / max_agents # normalize number of agents by maximal number of agents
        # compute utility as a linear combination of distance utility and crime utility, using weight
        utility = model.weight * c_utility + (1 - model.weight) * d_utility
        return utility
    
    def criminal_step(self):
        '''step method for criminal agents - 
        move to a random building based on probabilities, randomly select an agent and steal 1 resource point'''
        buildings = [b for b in model.buildings if b != self.building] # get all buildings but current
        max_agents = max([len(b.agents) for b in buildings]) # find the maximal number of agents per building
        utilities = [self.criminal_utility(b, max_agents) for b in buildings] # compute utility value per building
        if sum(utilities) > 0: # choices require that the sum of weights would be greater than 0
            new_building = choices(buildings, utilities)[0] # randomly (with weights based on probabilities) choose one building
        else: # if utilities sum to zero
            new_building = choice(buildings) # choose a random building
            
        if new_building.agents: # if there are any agents in the building (empty list==False)
            agent = choice(new_building.agents) # randomly choose an agent present in the building
            if agent.budget > 0: # if the agent has any resources - steal one unit
                agent.budget -= 1 # remove one unit of resources from selected agent
                self.budget += 1 # add one unit to current agent
                new_building.crimes += 1 # increase the new building's crime counter
    
        return new_building
    
    def step(self):
        '''General step method for both types of agents:
            locates new building and makes the required changes'''
        if self.a_type == 'resident': # check agent type and activate step method
            new_building = self.resident_step()
        else:
            new_building = self.criminal_step()
        
        self.building.agents.remove(self) # remove current agent from current building
        new_building.agents.append(self) # add current agent to new building
        self.building = new_building # move current agent to new building
        

class Model:
    def __init__(self, residents_num, criminal_num, weight, bldgs, connections):
        '''inputs: residents_num (int) - number of resident agents,
        criminal_num (int) - number of criminal agents,
        weight (float) - weight parameter of crime factor in utility function
        bldgs (GeoDataFrame object) - geodata on buildings
        connections (int) - number of other agents an agent knows'''
        
        self.weight = weight
        
        # create a list of building objects
        self.buildings = [Building(row.geometry) for idx, row in bldgs.iterrows()]
        # create a 1d Array of Agent objects  (length==resident_num) located in a random building and with a random budget
        self.residents = [Agent('resident', choice(self.buildings), randint(50, 800)) for i in range(residents_num)]
        # create a 1d Array of Agent objects  (length==criminal_num) located in a random building and with budget==50
        self.criminals = [Agent('criminal', choice(self.buildings), 50) for i in range(criminal_num)]
        
        # inititate social network
        self.network = nx.Graph()
        
        self.network.add_nodes_from(self.residents) # add resident agents to network
        
        for a in self.network.nodes: # for every resident agent = for every node
            candidates = [f for f in self.network.nodes if f != a] # find all agents that are not a
            distances = [a.building.distance(f.building) for f in candidates] # compute distance from agents
            sorted_distances = sorted(distances) # sort distances from lowest to highest
            ranks = [sorted_distances.index(d) for d in distances] # get a rank score for each distance
            # choose only agents whose rank is lower than connections
            friends = [candidates[i] for i in range(len(candidates)) if ranks[i] < connections]
            for f in friends:
                self.network.add_edge(a, f) # add an edge to the network between agent and friend
        
        # add loop which will eliminate 10% of edges and replace with an edge to a random node in network
        for e in list(self.network.edges):
            # draws random number between 0 and 1
            rand = random.random()
            # if drawn number is less than 0.1
            if rand < 0.1:
                n = e[0] 
                new_nodes_list = [n1 for n1 in self.network.nodes if n1 != n and n1 not in self.network.neighbors(n)]
                # choose new neighbor
                new_neigh = random.choice(new_nodes_list)
                # add edge to new neighbor and remove the edge to old neighbor
                self.network.add_edge(n, new_neigh)
                self.network.remove_edge(e[0], e[1])
                
    def simulate(self):
        '''Run 30 iterations and return the total number of crimes, maximal number of crimes per building,
        and percent of buildings where no crimes were registered througout the simulation'''
        for i in range(30):
            agents = self.residents + self.criminals # merge the lists of agents into one
            shuffle(agents) # randomly reorder the merged list
            for agent in agents: # execute step function per agent
                agent.step()
        
        # generate outputs - get the count of crimes per building
        crimes = [b.crimes for b in self.buildings]
        total_crimes = sum(crimes) # total number of crimes committed throughout the simulation
        max_crimes = max(crimes) # get the counter value for the building with the most crimes committed
        # compute the percent of buildings where no crime was committed
        no_crimes = 100 * len([c for c in crimes if c==0]) / len(self.buildings)
        return [total_crimes, max_crimes, no_crimes]
        

criminal_num = 40
weight = 0.5
buildings = gpd.read_file('/Users/noamgal/Downloads/bldgs_points(1)/bldgs_points.shp')
connections = 3
model = Model(200-criminal_num, criminal_num, weight, buildings, connections)
model.simulate()

[126, 14, 92.00477326968974]

In [16]:
# Reusing the same parameters and code from part B, but with the updated simulate function
residents_num = 160
criminal_num = 40
weights = [0.25, 0.5, 0.75]
connections = [1,2,3]
# Create dataframe for Part C results
partC_df = pd.DataFrame(columns=['residents_num', 'criminal_num', 'weight', 'connections', 'result_total', 'result_max', 'result_percent'])

# Iterates through the assigned parameters
for c in connections:
    for w in weights:
        result_total = []
        result_max = []
        result_percent = []
        # Runs five times for each parameter set and adds results to a list
        for run in range(5):
            model = Model(residents_num, criminal_num, w, buildings, c)
            results = model.simulate()
            #print(results)
            result_total.append(results[0])
            result_max.append(results[1])
            result_percent.append(results[2])
            
        #print(f'The average total for weight {w} and connection {c} is:',np.mean(result_total))
        #print(f'The average max for weight {w} and connection {c} is:',np.mean(result_max))
        #print(f'The average percent for weight {w} and connection {c} is:',np.mean(result_percent))
        
        # Averages the results for each set of parameters and adds to a dataframe
        new_row = {
            'residents_num': residents_num,
            'criminal_num': criminal_num,
            'weight': w,
            'connections': c,
            'result_total': np.mean(result_total),
            'result_max': np.mean(result_max),
            'result_percent': np.mean(result_percent)}
        partC_df.loc[len(partC_df)] = new_row
        
# Export dataframe of results
#partC_df.to_csv('SimEx2-partC.csv')

## Statistical Analysis of results from Part C

In [18]:
# List of predictor columns
predictors = ['weight', 'connections']

# List of result columns
results = ['result_total', 'result_max', 'result_percent']

# Perform regression between each predictor and each result column
for predictor in predictors:
    for result in results:
        # Reusing a regression function defined in Part B
        perform_regression(partC_df, predictor, result)

weight vs result_total: R-squared = 0.22, p-value = 0.1992
weight vs result_max: R-squared = 0.84, p-value = 0.0005
weight vs result_percent: R-squared = 0.05, p-value = 0.5580
connections vs result_total: R-squared = 0.59, p-value = 0.0161
connections vs result_max: R-squared = 0.00, p-value = 0.8711
connections vs result_percent: R-squared = 0.77, p-value = 0.0019


In [32]:
partC_df.corr()

,residents_num,criminal_num,weight,connections,result_total,result_max,result_percent
residents_num,NaN,NaN,NaN,NaN,NaN,NaN,NaN
criminal_num,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weight,NaN,NaN,1.000000e+00,-3.700743e-17,0.472368,0.917416,-0.226415
connections,NaN,NaN,-3.700743e-17,1.000000e+00,-0.766133,-0.063469,0.877736
result_total,NaN,NaN,4.723680e-01,-7.661327e-01,1.000000,0.612130,-0.950502
result_max,NaN,NaN,9.174163e-01,-6.346905e-02,0.612130,1.000000,-0.366148
result_percent,NaN,NaN,-2.264152e-01,8.777361e-01,-0.950502,-0.366148,1.000000
